Nova tentativa


Mudança na verificação dos contours, porque em alguns casos a countour do tabuleiro não é representado como um quadrilatero. :(

In [ ]:
import cv2
import numpy as np



def print_resized(name, img):
    resize_img = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    cv2.imshow(name, resize_img)



def detect_game_board(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not load image.")
        return None, None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0) # to remove some noise
    edges = cv2.Canny(blurred, 50, 150)
    kernel = np.ones((3, 3), np.uint8)

    print_resized("Canny Edges", edges)

    dilated_edges = cv2.dilate(edges, kernel, iterations=1) # dilate to make them more visible

    print_resized("Canny Dilated", dilated_edges)

    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    board_contour = None
    max_area = 0
    i = 0
    for contour in contours:
        # Get convex hull because it works better

        hull = cv2.convexHull(contour)
        peri = cv2.arcLength(hull, True)
        approx = cv2.approxPolyDP(hull, 0.05 * peri, True)  # Higher tolerance for better board detection
        
        # If is has 4 sides
        if len(approx) == 4:
            i += 1
            print("Contour" + str(i))
            area = cv2.contourArea(contour)
            if area > max_area:
                max_area = area
                board_contour = approx
                

    if board_contour is None:
        print("No game board detected.")
        return image, None

    corners = board_contour.reshape(4, 2).astype(np.float32) #get the corners of the board
    width, height = 500, 500
    dst_points = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], dtype=np.float32)
    matrix = cv2.getPerspectiveTransform(corners, dst_points)
    warped = cv2.warpPerspective(image, matrix, (width, height))

    cv2.drawContours(image, [board_contour], -1, (0, 255, 0), 3)
    return image, warped


# For testing
image_path = "images\G000_IMG062.jpg"
#image_path = "images\G000_IMG087.jpg"
#image_path = "images\G000_IMG102.jpg"
#image_path = "images\G006_IMG048.jpg"
#image_path = "images\G006_IMG086.jpg"
#image_path = "images\G006_IMG119.jpg"
#image_path = "images\G019_IMG082.jpg"
#image_path = "images\G028_IMG015.jpg" #aqui muito fora
#image_path = "images\G028_IMG062.jpg"
#image_path = "images\G028_IMG098.jpg" #aqui ligeiramente
#image_path = "images\G028_IMG101.jpg"
#image_path = "images\G033_IMG043.jpg"
#image_path = "images\G033_IMG075.jpg"
#image_path = "images\G033_IMG088.jpg"
#image_path = "images\G033_IMG101.jpg"
#image_path = "images\G038_IMG074.jpg"
#image_path = "images\G038_IMG088.jpg"
#image_path = "images\G038_IMG103.jpg"
#image_path = "images\G038_IMG105.jpg" #mais para dentro
#image_path = "images\G041_IMG042.jpg"
#image_path = "images\G041_IMG048.jpg"
#image_path = "images\G041_IMG088.jpg"
#image_path = "images\G041_IMG098.jpg"
#image_path = "images\G047_IMG053.jpg"
#image_path = "images\G047_IMG068.jpg"
#image_path = "images\G047_IMG102.jpg"
#image_path = "images\G047_IMG107.jpg" #aqui ligeiramente
#image_path = "images\G056_IMG017.jpg"
#image_path = "images\G056_IMG077.jpg" #aqui ligeiramente
#image_path = "images\G056_IMG097.jpg" #aqui ligeiramente
#image_path = "images\G058_IMG044.jpg"
#image_path = "images\G058_IMG074.jpg"
#image_path = "images\G058_IMG100.jpg"
#image_path = "images\G061_IMG080.jpg" #mais para dentro
#image_path = "images\G061_IMG092.jpg"
#image_path = "images\G061_IMG098.jpg"
#image_path = "images\G072_IMG083.jpg" #aqui
#image_path = "images\G072_IMG098.jpg"
#image_path = "images\G076_IMG072.jpg"
#image_path = "images\G076_IMG089.jpg"
#image_path = "images\G076_IMG095.jpg"
#image_path = "images\G078_IMG092.jpg"
#image_path = "images\G083_IMG073.jpg"
#image_path = "images\G083_IMG089.jpg"
#image_path = "images\G087_IMG093.jpg" #aqui
#image_path = "images\G087_IMG099.jpg"
#image_path = "images\G091_IMG053.jpg"
#image_path = "images\G091_IMG074.jpg"
#image_path = "images\G091_IMG102.jpg"
#image_path = "images\G099_IMG094.jpg"



img_copy = cv2.imread(image_path)
print_resized("Original Image", img_copy)

original_with_contour, corrected_board = detect_game_board(image_path)
if original_with_contour is not None:
    cv2.imshow("Original with Contour", original_with_contour)
    if corrected_board is not None:
        cv2.imshow("Corrected Board", corrected_board)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Contour1
Contour2
Contour3
Contour4
Contour5
Contour6
Contour7
Contour8
Contour9
Contour10
Contour11
Contour12
Contour13
Contour14


In [5]:
# Try for the hough lines



cv2.imshow("board", corrected_board)
#print_resized("Original Image", corrected_board)

#detect lines
gray = cv2.cvtColor(corrected_board, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)
cv2.imshow("Canny Edges", edges)

lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)  # Threshold 150, adjust as needed

# print the lines

if lines is not None:
    for rho, theta in lines[:, 0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
        cv2.line(corrected_board, pt1, pt2, (0, 0, 255), 1)

cv2.imshow("Hough Lines", corrected_board)




cv2.waitKey(0)
cv2.destroyAllWindows()

representar em matriz ???

In [6]:
cv2.imshow("board",corrected_board)

cv2.waitKey(0)
cv2.destroyAllWindows()